In [96]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date
hoje =  pd.to_datetime(date.today())

In [97]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 11/01/23 - Horário: 09:30:27.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 11/01/23 - Horário: 09:33:34.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 11/01/23 - Horário: 09:33:38.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 11/01/23 - Horário: 09:33:50.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 11/01/23 - Horário: 09:34:56.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 11/01/23 - Horário: 09:36:08.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 11/01/23 - Horário: 09:36:21.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 11/01/23 - Horário: 09:37:06.  Portanto não foi baixado novamente.
vrapeelo

In [98]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
ide_data_cols = ['DthEnvio','IdeUsinaOutorga']
ide_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Baixa bancos de dados

In [99]:
# Carrega BD vmonitoramentoleilao => monitoramentoleilao
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao.sample(5)

,IdeUsinaOutorga,CodLeilao,DatInicioSuprimento
3029,55707,01/2021,2022-05-01
110,30675,03/2022,2026-01-01
950,37820,03/2018,2024-01-01
755,37294,03/2018,2024-01-01
80,30054,11/2021,2026-07-01


In [100]:
# Carrega BD vrapeelcronograma => rapeel_cronograma
cols = ['IdeUsinaOutorga','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DthEnvio' : 'Data_Envio',
        'NomUsina' : 'usina_nome',
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma.sample(5)

,IdeUsinaOutorga,Data_Envio,canteiroReal,prev_IO,IO_real,DesvRio_real,Ench_Real,IOTrans_Real,Conc_Trans_REAL,Comiss_Real,ME_Real_conc_eol,Conc_Combust_Real,DatRealizacaoVI,DatRealizacaoX,DatRealizacaoV,DatRealizacaoIV
3307,37952,2020-08-04 13:54:43.417,2019-10-07,NaT,2020-02-10,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2020-07-06,NaT
5315,43273,2020-10-05 22:52:07.387,NaT,2022-01-24,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
24566,48641,2022-05-02 14:11:15.297,NaT,2022-06-25,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
32302,43270,2022-09-05 11:50:46.937,NaT,2024-01-24,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4990,37494,2020-10-05 14:45:03.040,NaT,2020-12-31,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [101]:
# Carrega BD vrapeelcontratorecurso => mercado

cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)

# Carrega BD vrapeellicenciamento => ambiental_datas
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeelacesso => acesso_contratos
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeeloperacaoug => ug_rapeel
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ug_rapeel.loc[ug_rapeel.groupby(['IdeUsinaOutorga','NumOperacaoUg']).DthEnvio.idxmax()]

# Cria dataframe input_rapeel

In [102]:
# Carrega BD vrapeelacesso => acesso_contratos
input_rapeel = pd.merge(mercado,ambiental_datas,on=ide_data_cols,how="inner")
input_rapeel = pd.merge(input_rapeel,acesso_contratos,on=ide_data_cols,how="inner")
input_rapeel = input_rapeel.loc[input_rapeel.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
input_rapeel.sample(5)

,DthEnvio,IdeUsinaOutorga,DatConclusaoACL,DatValidadeLI,DatValidadeLP,DatValidadeLO,NomSitContratoCCD,NomSitContratoCUST,NomSituacaoContratoCUSD,NomSitContratoCCT
254,2021-11-05 15:58:08.933,35699,NaT,2025-02-02,2020-10-10,2027-09-16,Não se Aplica,Válido,Não se Aplica,Válido
666,2023-01-02 10:47:49.023,54639,NaT,2032-01-31,2032-01-31,2032-01-31,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado
1215,2023-01-02 14:41:45.350,49857,NaT,NaT,2026-04-06,NaT,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado
340,2022-03-04 15:19:59.683,31408,NaT,2025-06-12,2023-07-17,2026-11-24,Não se Aplica,Válido,Não se Aplica,Válido
334,2022-02-15 12:44:26.440,37069,2018-06-14,2022-08-19,2020-09-28,NaT,Válido,Não se Aplica,Válido,Não se Aplica


In [103]:
definicoes =   input_rapeel #pd.merge(input_rapeel,ug_rapeel,on=ide_data_cols,how='inner')
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

definicoes['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    definicoes.loc[definicoes.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = definicoes[validade]
definicoes[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5)

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
977,49835,NaT,NaT,2022-12-30,2022-12-30
212,33606,2024-12-10,2021-03-07,2019-02-01,2024-12-10
496,31686,2023-05-20,2016-12-28,2013-11-01,2023-05-20
2375,54843,NaT,NaT,NaT,NaT
690,54654,2032-01-31,2032-01-31,2032-01-31,2032-01-31


In [104]:
cols = ["IdeUsinaOutorga","DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","DatMonitoramento","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado"]

monitoramentodados = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]

In [105]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao
rename_cols = {'DatPrevistaComercial':'DatPrevistaComercial'}
definicoes_monitoramento = pd.merge(monitoramentodados,definicoes,on='IdeUsinaOutorga',how="left").rename(columns=rename_cols).drop_duplicates()
definicoes_monitoramento = definicoes_monitoramento.loc[definicoes_monitoramento.IdcUsinaMonitorada == "Sim"]

definicoes_monitoramento['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    definicoes_monitoramento.loc[definicoes_monitoramento[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()) & (definicoes_monitoramento.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


definicoes_monitoramento[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(definicoes_monitoramento[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

definicoes_monitoramento['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSitContratoCCD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCCT == nsa) &
    definicoes_monitoramento.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

definicoes_monitoramento.loc[
(    ((definicoes_monitoramento.NomSitContratoCCD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCCT == valido)) &
    definicoes_monitoramento.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSitContratoCCD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCCT == vencido) |
    (definicoes_monitoramento.NomSitContratoCCD == nass) |
    (definicoes_monitoramento.NomSitContratoCCT == nass )) &
    definicoes_monitoramento.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


definicoes_monitoramento['CondicaoUso'] = "Não informado"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSituacaoContratoCUSD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCUST == vencido) |
    (definicoes_monitoramento.NomSituacaoContratoCUSD == nass) |
    (definicoes_monitoramento.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


definicoes_monitoramento["PPA"] = "Ambos"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
0,NaT,2020-12-11,2017-09-18,LI,2020-12-11
1,NaT,2026-03-09,2002-12-11,LI,2026-03-09
2,NaT,NaT,NaT,Sem LP,NaT
3,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
7,NaT,NaT,NaT,Não Informado,NaT
...,...,...,...,...,...
3093,NaT,NaT,NaT,Não Informado,NaT
3094,NaT,NaT,NaT,Não Informado,NaT
3095,NaT,NaT,NaT,Não Informado,NaT
3096,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
846,37636,Não se Aplica,Não Assinado,2023-01-05 16:24:56.143,Sem Conexão
1597,46561,Não se Aplica,Válido,2023-01-04 19:18:09.973,OK
1713,47208,Não se Aplica,Não Assinado,2023-01-04 14:10:16.880,Sem Conexão
1629,46736,Não se Aplica,Válido,2023-01-01 19:38:49.697,OK
931,37783,Não se Aplica,Válido,2023-01-04 18:45:05.883,OK
2424,50195,Não se Aplica,Válido,2023-01-05 20:05:01.480,OK
374,33848,Não Assinado,Não Assinado,2022-01-03 14:34:16.333,Sem Conexão
2765,52391,Não se Aplica,Não Assinado,2023-01-05 16:19:26.640,Sem Conexão
564,35214,Não se Aplica,Válido,2023-01-04 10:57:32.783,OK
2808,53841,Não se Aplica,Não Assinado,2023-01-02 11:27:49.817,Sem Conexão


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
3011,55740,Válido,Válido,Verificar
71,29576,Válido,Não se Aplica,OK
2116,49398,Não se Aplica,Válido,OK
1732,47237,Não se Aplica,Válido,OK
1749,47254,Não se Aplica,Não Assinado,Sem Conexão
1554,46094,Não se Aplica,Não Assinado,Sem Conexão
2514,50870,Não se Aplica,Válido,OK
383,33883,Válido,Não se Aplica,OK
210,32237,NaN,NaN,Não informado
586,35309,Válido,Não se Aplica,Verificar


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
2622,51646,Fora do ACR,NaT,Nenhum
1166,40690,Fora do ACR,NaT,Nenhum
226,32398,Fora do ACR,NaT,Nenhum
1610,46583,Fora do ACR,NaT,Nenhum
474,34629,Fora do ACR,NaT,Nenhum
2448,50473,Fora do ACR,NaT,Nenhum
1901,47474,Fora do ACR,NaT,Nenhum
1606,46577,Fora do ACR,NaT,Nenhum
2908,54644,Fora do ACR,NaT,Nenhum
2156,49445,Fora do ACR,NaT,Nenhum


In [106]:
# Apenas mudança de nome
tabela_criterio = definicoes_monitoramento

In [107]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

tabela_criterio['criterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


tabela_criterio['dscriterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
1842,47388,Não Iniciada,Nenhum,LP,OK,4,"Sem obras, sem PPA e LP OK e Cust Ok"
1360,44439,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2824,53883,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2593,51553,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
1793,47328,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
3064,57328,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2651,51785,Em andamento,Nenhum,LI,OK,0,Usina em Obras em andamento
3086,70551,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
929,37773,Não Iniciada,ACL,LI,OK,1,Sem obras e PPA Ok
1192,40755,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"


In [108]:
slp = "Sem LP"
sim = "Sim"
tabela_criterio['criterio_novo'] = np.select(
    [
        (tabela_criterio.IdcSemPrevisao == sim),
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        (tabela_criterio.CondicaoAmbiental == "Sem LP"),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
2034,49144,Não Iniciada,Nenhum,LP,OK,4,"Sem obras, sem PPA e LP OK e Cust Ok"
903,37726,Paralisada,ACR,LI,Verificar,9,Obras paralisadas
2544,50956,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
687,35851,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1481,45025,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"


In [109]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100


In [110]:
cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)

for col in ['SigTipoGeracao','DatMonitoramento ']:
    if 'SigTipoGeracao' in calculo_previsao.columns:
        calculo_previsao.drop(['SigTipoGeracao'],inplace=True,axis=1)

### criterios_da_previsao

In [115]:
criterios_da_previsao  = pd.merge(
                                    tabela_criterio,calculo_previsao,
                                    on="IdeUsinaOutorga",how='inner'
                                    )
criterios_da_previsao = pd.merge(criterios_da_previsao,ug_rapeel,left_on= ['IdeUsinaOutorga','NumUgUsina'],right_on=['IdeUsinaOutorga','NumOperacaoUg'],how='left')

del criterios_da_previsao['NumOperacaoUg']

# Regras

In [116]:
################################ regra_antiga ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regra_antiga'] = pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regra_antiga'] = criterios_da_previsao[['Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regra_antiga'] = ( hoje +pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regra_antiga'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] >= 3) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[criterios_da_previsao['criterio'] >= 3 & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)


################################ regrapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regrapmo'] =  pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regrapmo'] = ( hoje + pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regrapmo'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8) ),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(3*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(4*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5) | (criterios_da_previsao['criterio'] == 7),'regrapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'regrapmo'] = pd.NA

In [117]:
################################ dscjustificativapmo ################################
list_condicoes = [
    (criterios_da_previsao['IdcSemPrevisao'] == "Sim"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT") & ( criterios_da_previsao['Previsao_OC'] > criterios_da_previsao['DatPrevistaComercial']),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT") & ( criterios_da_previsao['Previsao_OC'] < criterios_da_previsao['DatPrevistaComercial']),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),
    ( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Dat_OC_obrigacao']),
    ( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Dat_OC_obrigacao']),
    ( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")) & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Previsao_OC']),
    ( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")) & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Previsao_OC']), 
    (criterios_da_previsao['criterio'] == 3),
    (criterios_da_previsao['criterio'] == 4) | (criterios_da_previsao['criterio'] == 5),
    (criterios_da_previsao['criterio'] == 7),
    (criterios_da_previsao['criterio'] == 6),
]

list_values = [
    "Analisar justificativa",
    "Estágio Atual das Obras",
    "Estágio Atual das Obras",
    "Usina liberada para testes",
    "Testes não concluídos",
    "Paralisação de Obras",
    "Paralisação de Obras",
    "Estágio Atual das Obras",
    "Estágio Atual das Obras" ,
    "Não celebrado contrato de uso do sistema de transmissão/distribuição",
    "Situação da Licença de Instalação",
    "Sem Rapeel",
    "Sem Licenciamento Ambiental"
]

criterios_da_previsao['dscjustificativapmo'] = np.select(list_condicoes,list_values,default="Null")


################################ criterionovopmo ################################
list_condicoes = [
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT")),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT")),
    ((criterios_da_previsao['criterio_novo'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 2)),
    ((criterios_da_previsao['criterio_novo'] == 3)),
    ((criterios_da_previsao['criterio_novo'] == 5)),
    ((criterios_da_previsao['criterio_novo'] == 4)),
    ((criterios_da_previsao['criterio_novo'] == 6)),
    ((criterios_da_previsao['criterio_novo'] == 7)),
    ((criterios_da_previsao['criterio_novo'] == 8)),
    ((criterios_da_previsao['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

criterios_da_previsao['criterionovopmo'] = np.select(list_condicoes,list_values)



criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 9),'dsccriterionovo'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterionovo'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterionovo'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterionovo'] = "Usina em Teste = Previsão OC conforme data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 1),'dsccriterionovo'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 2),'dsccriterionovo'] = "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 3),'dsccriterionovo'] = "Usina sem obras, sem cust, sem PPA e com LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 4),'dsccriterionovo'] = "Usina sem obras, com cust, sem PPA e sem LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 5),'dsccriterionovo'] = "Usina sem obras, sem cust, sem PPA e sem LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 7),'dsccriterionovo'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 6),'dsccriterionovo'] = "Usina sem LP = Previsão OC sem previsão"
criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 8),'dsccriterionovo'] = "Usina obras paralisadas == Previsão OC maior entre data RAPEEL e data compromisso"

In [118]:
################################ regranovapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regranovapmo'] = pd.NA

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].isna()),'regranovapmo'] = criterios_da_previsao['DatPrevistaComercial']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].notna()),'regranovapmo'] = criterios_da_previsao['Previsao_OC']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo']== 1) | (criterios_da_previsao['criterio_novo']== 2) ,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')

criterios_da_previsao.loc[((criterios_da_previsao['criterio_novo'] == 3) | (criterios_da_previsao['criterio_novo'] == 4) | (criterios_da_previsao['criterio_novo'] == 5) ),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 6,'regranovapmo'] = pd.NA

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(6*365,'D')
criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 7,'regranovapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 8,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)


################################ criterio_pmo_ts ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'criterio_pmo_ts'] = 8

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'criterio_pmo_ts'] = 0.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'criterio_pmo_ts'] = 0.2

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'criterio_pmo_ts'] = 0.3

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'criterio_pmo_ts'] = 1.1

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'criterio_pmo_ts'] = 1.2

criterios_da_previsao.loc[((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'criterio_pmo_ts'] = 2.2

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),'criterio_pmo_ts'] = 2.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'criterio_pmo_ts'] = 3

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'criterio_pmo_ts'] = 4

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5),'criterio_pmo_ts'] = 5

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'criterio_pmo_ts'] = 7

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'criterio_pmo_ts'] = 6

In [119]:
################################ dsccriterio_pmo_ts ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias == Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste == Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste = Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'dsccriterio_pmo_ts'] = "Usina sem PPA com LI sem CUST = Previsão OC maior entre handicap de 3 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI com CUST = Previsão OC maior entre handicap de 4 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI sem CUST = Previsão OC maior entre handicap de 5 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'dsccriterio_pmo_ts'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC maior entre handicap de 5 anos e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'dsccriterio_pmo_ts'] = "Usina sem LP = Previsão OC sem previsão"

In [121]:
################################ criterio_pmo_ts_max_usina ################################
criterio_pmo_ts_max = criterios_da_previsao.groupby("IdeUsinaOutorga")[['criterio_pmo_ts']].max().rename(columns=
{'criterio_pmo_ts': 'criterio_pmo_ts_max_usina'})
criterios_da_previsao = criterios_da_previsao.merge(criterio_pmo_ts_max,on="IdeUsinaOutorga",how='left')


################################ dscjustificativaregra ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dscjustificativaregra'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

criterios_da_previsao.loc[(criterios_da_previsao['IdcObraParalisada'] == "Sim") & (criterios_da_previsao['IdcSemPrevisao'] != "Sim"),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.2),'dscjustificativaregra'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),'dscjustificativaregra'] = "Estágio das obras da usina"


criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.1),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.2),'dscjustificativaregra'] = "Compromisso de venda de energia - PPA"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 2.2),'dscjustificativaregra'] = "Licença de Instalação - LI e acesso a rede - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 3),'dscjustificativaregra'] = "Acesso a rede contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 4),'dscjustificativaregra'] = "Acesso a rede não contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 5),'dscjustificativaregra'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 6),'dscjustificativaregra'] = "Nenhuma licença ambiental válida"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 7),'dscjustificativaregra'] = "Sem RAPEEL"


################################ dscjustificativaregranova ################################
criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 9),'dscjustificativaregranova'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 8),'dscjustificativaregranova'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 0.2),'dscjustificativaregranova'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),'dscjustificativaregranova'] = "Estágio das obras da usina"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 1),'dscjustificativaregranova'] = "Compromisso de venda de energia - PPA"

criterios_da_previsao.loc[criterios_da_previsao['criterionovopmo']==2,'dscjustificativaregranova'] = "Acesso a rede contratado - CUST/CUSD"

criterios_da_previsao.loc[criterios_da_previsao['criterionovopmo']==3,'dscjustificativaregranova'] = "Acesso a rede não contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 4),'dscjustificativaregranova'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 5),'dscjustificativaregranova'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 6),'dscjustificativaregranova'] = "Nenhuma licença ambiental válida"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 7),'dscjustificativaregranova'] = "Sem RAPEEL"

In [122]:
#criterios_da_previsao_sas = pd.read_excel("./WORK_Criteriosdaprevisao.xlsx")
criterios_da_previsao_sas['SAS'] = True
criterios_da_previsao_sas.CalculoPrevisorOC_mx = criterios_da_previsao_sas.CalculoPrevisorOC_mx.dt.normalize()

In [123]:
criterios_da_previsao["PY"] = True

In [146]:
merge = pd.merge(criterios_da_previsao,criterios_da_previsao_sas,on=['IdeUsinaOutorga','NumUgUsina'],suffixes=('_py','_sas'),how='outer')
merge['OC_diff'] = merge.Previsao_OC - merge.CalculoPrevisorOC_mx

# Teste previsão

In [177]:
merge[['IdeUsinaOutorga','NumUgUsina'] + [ 'Previsao_OC','CalculoPrevisorOC_mx','OC_diff','IdcUsinaMonitorada']].sort_values(by='OC_diff',ascending=False).head(10)

,IdeUsinaOutorga,NumUgUsina,Previsao_OC,CalculoPrevisorOC_mx,OC_diff,IdcUsinaMonitorada
142,30414,1,2025-03-18,2022-09-08,922 days,Sim
148,30612,1,2025-03-29,2023-01-23,796 days,Sim
237,31691,6,2023-08-27,2022-06-04,449 days,Sim
236,31691,3,2023-08-27,2022-06-04,449 days,Sim
242,31691,7,2023-08-27,2022-06-04,449 days,Sim
241,31691,4,2023-08-27,2022-06-04,449 days,Sim
240,31691,1,2023-08-27,2022-06-04,449 days,Sim
239,31691,8,2023-08-27,2022-06-04,449 days,Sim
238,31691,5,2023-08-27,2022-06-04,449 days,Sim
235,31691,2,2023-08-27,2022-06-04,449 days,Sim


In [175]:
calculo_previsao[calculo_previsao.Previsao_OC.isna()]

,NomUsina,IdeUsinaOutorga,NumUgUsina,Previsao_OC,Dat_OC_obrigacao,DatMonitoramento,FASE,Ind_crono_norm,flagOPTeste30dias,DatPrevisaoIniciobra,DatInicioObraOutorgado
0,Caldeirões,601,2,NaT,2003-10-31,2022-10-13,OUT,NaN,0,NaT,2001-01-30
1,Melo Viana,1455,2,NaT,2008-12-01,2022-10-13,OUT,NaN,0,NaT,2007-05-01
8,Apertadinho,28492,2,NaT,2004-10-30,2022-10-13,OUT,NaN,0,NaT,2002-11-15
9,Palma,28785,2,NaT,2006-01-31,2022-10-13,OUT,NaN,0,NaT,2004-04-30
11,Primavera do Rio Turvo,29177,2,NaT,2007-12-30,2022-10-13,OUT,NaN,0,NaT,2006-05-01
...,...,...,...,...,...,...,...,...,...,...,...
73163,Turvânia VII,54847,1,NaT,2027-04-11,2022-12-12,OUT,NaN,0,NaT,NaT
73164,Turvânia VII,54847,4,NaT,2027-04-11,2022-12-12,OUT,NaN,0,NaT,NaT
73165,Turvânia VII,54847,7,NaT,2027-04-11,2022-12-12,OUT,NaN,0,NaT,NaT
73166,Turvânia VIII,54848,11,NaT,2027-10-15,2022-12-12,OUT,NaN,0,NaT,NaT


In [172]:
merge[merge.Previsao_OC.isna()]

,IdeUsinaOutorga,DatCanteiroObraRealizado,DatDesvioRioRealizado,DatEnchimentoRealizado,DatConclusaoSisTransRealizado,DatPrevisaoIniciobra_x,IdcObraParalisada_py,IdcUsinaMonitorada,DatMontagemOutorgado,DatConcretagemRealizado,DatConclusaoTorresRealizado,DatInicioObraOutorgado_x,DatConcretagemOutorgado,DatSisTransmissaoRealizado,IdcSituacaoObra_py,IdcSemPrevisao,DatComissionamentoUGRealizado,DatMonitoramento_x,SigTipoGeracao_py,DscComercializacaoEnergia,DatInicioObraRealizado,DatMontagemRealizado,DthEnvio_x,DatConclusaoACL,DatValidadeLI,DatValidadeLP,DatValidadeLO,NomSitContratoCCD,NomSitContratoCUST,NomSituacaoContratoCUSD,NomSitContratoCCT,ValidadeAmbiental_py,CondicaoAmbiental_py,CondicaoConexao,CondicaoUso_py,PPA_py,criterio,dscriterio,criterio_novo,NomUsina_py,NumUgUsina,Previsao_OC,Dat_OC_obrigacao,DatMonitoramento_y,FASE_py,Ind_crono_norm_py,flagOPTeste30dias_py,DatPrevisaoIniciobra_y,DatInicioObraOutorgado_y,DthEnvio_y,...,RegraAntiga,RegraPMO,SigTipoGeracao_sas,NomUsina_sas,UFUsina,NomComplexo,IdcComplexo,MdaPotenciaUnitaria,DscJustificativaPMO,DscJustificativaPrevisaoAtual,IdcObraParalisada_sas,IdcSituacaoObra_sas,IdcViabilidade,DscSistema,PPA_sas,CondicaoUso_sas,CondicaoAmbiental_sas,ValidadeAmbiental_sas,DatPrevisaoIniciobra,DatInicioObraOutorgado,CalculoPrevisorOC_mx,Previsão_atual_mx,OC_Obrigacao_mx,DatPrevistaComercialRapeelMX,DatMonitoramento,DthEnvio,FASE_sas,DscObservacaoUG,DscSolucao,IdcTipoEntrave,DscObservacaoAcesso,DscObservacaomercado,DscObservacaoAmbiental,cont,flagOPTeste30dias_sas,Valor_Semaforico,Ind_crono_mn,Ind_crono_norm_sas,IOprev_rapeel,IOreal_rapeel,DatEmissaoAto,CriterioNovoPMO,DscCriterioNovo,RegraNovaPMO,CriterioPMO_TS,DscCriterioPMO_TS,DscJustificativaRegra,DscJustificativaRegranova,SAS,OC_diff
2,324,NaT,NaT,NaT,NaT,NaT,Não,Sim,2015-03-31,NaT,NaT,2014-02-28,2014-12-31,NaT,Não Iniciada,Sim,NaT,2022-10-13 11:01:56.560,UHE,Fora do ACR,NaT,NaT,2023-01-02 08:34:07.347,NaT,2026-03-09,2002-12-11,NaT,Não se Aplica,Não se Aplica,Válido,Válido,2026-03-09,LI,OK,OK,Nenhum,2,"Sem obras, sem PPA e LI Ok e Cust OK",9,Bugres,1,NaT,2015-06-30,2022-10-13,OUT,NaN,0,NaT,2014-02-28,2023-01-02 08:34:07.347,...,NaT,NaT,UHE,Bugres,RS,NaN,NaN,7500.00,Analisar justificativa,Inviabilidade da implantação da usina,Não,Não Iniciada,Baixa,Sistema Interligado Nacional,Nenhum,OK,LI,2026-03-09,NaT,2014-02-28,2025-11-04,NaT,2015-06-30,2026-07-01 00:00:00,2022-10-13 11:01:56.560,2023-01-02 08:34:07.347,OUT,NaN,NaN,Não há pendências,Parecer de acesso emitido em 09/09/2022.,Assinado Contrato de Concessão n°004/2022.\nEm...,NaN,1,0,-3,9.999097e-01,9.033179e-05,2025-02-01 00:00:00,NaT,2022-11-18,9.0,Usina Viabilidade Baixa análise da fiscalização,NaT,8.0,Usina Viabilidade Baixa análise da fiscalização,Analisar justificativa: Revogação da outorga e...,Analisar justificativa: Revogação da outorga e...,True,NaT
3,601,NaT,NaT,NaT,NaT,NaT,Não,Sim,2001-02-15,NaT,NaT,2001-01-30,NaT,NaT,Não Iniciada,Sim,NaT,2022-10-13 11:03:48.340,PCH,Fora do ACR,NaT,NaT,2023-01-03 19:22:43.140,NaT,NaT,NaT,NaT,Não Assinado,Não se Aplica,Não Assinado,Não se Aplica,NaT,Sem LP,Sem Conexão,Sem Uso,Nenhum,6,Sem LP,9,Caldeirões,2,NaT,2003-10-31,2022-10-13,OUT,NaN,0,NaT,2001-01-30,2023-01-03 19:22:43.140,...,NaT,NaT,PCH,Caldeirões,MG,NaN,NaN,5000.00,Analisar justificativa,Nenhuma licença ambiental válida,Não,Não Iniciada,Baixa,Sistema Interligado Nacional,Nenhum,Sem Uso,Sem LP,NaT,NaT,2001-01-30,2024-11-08,NaT,2003-10-31,2027-05-01 00:00:00,2022-10-13 11:03:48.340,2023-01-03 19:22:43.140,OUT,NaN,Os documentos complementares para emissão da L...,LP da usina,.,NaN,O órgão ambiental antevendo a necessidade de a...,1,0,-3,1.000000e+00,5.095924e-14,2026-05-29 00:00:00,NaT,2000-08-30,9.0,Usina Viabilidade Baixa análise da fiscalização,NaT,8.0,Usina Viabilidade Baixa análise da fiscalização,Analisar justificativa: Revogação da outorga e...,Analisar justificativa: Revogação da outorga e...,True,NaT
4,601,NaT,NaT,NaT,NaT,NaT,Não,Sim,2001-02-15,NaT,NaT,2001-01-30,NaT,NaT,N

# Testes critérios

In [147]:
merge[merge.CriterioNovoPMO != merge.criterionovopmo][['IdeUsinaOutorga','NumUgUsina'] + ['flagOPTeste30dias_sas','flagOPTeste30dias_py','FASE_sas','FASE_py','criterio_novo','criterio','CriterioNovoPMO','criterionovopmo']]

,IdeUsinaOutorga,NumUgUsina,flagOPTeste30dias_sas,flagOPTeste30dias_py,FASE_sas,FASE_py,criterio_novo,criterio,CriterioNovoPMO,criterionovopmo


In [163]:
merge.OC_diff.max()

Timedelta('922 days 00:00:00')

,IdeUsinaOutorga,NumUgUsina,IdcSemPrevisao,IdcObraParalisada_sas,FASE_sas,criterio,DatPrevistaComercial,Previsao_OC,CalculoPrevisorOC_mx,OC_diff,DscJustificativaPMO,dscjustificativapmo
142,30414,1,Não,Não,OUT,3,2025-05-30,2025-03-18,2022-09-08,922 days,Não celebrado contrato de uso do sistema de tr...,Não celebrado contrato de uso do sistema de tr...


In [149]:
merge[merge.DscJustificativaPMO  != merge.dscjustificativapmo ][['IdeUsinaOutorga','NumUgUsina'] +['IdcSemPrevisao','IdcObraParalisada_sas','FASE_sas','criterio','DatPrevistaComercial','Previsao_OC','CalculoPrevisorOC_mx','DscJustificativaPMO','dscjustificativapmo']]

,IdeUsinaOutorga,NumUgUsina,IdcSemPrevisao,IdcObraParalisada_sas,FASE_sas,criterio,DatPrevistaComercial,Previsao_OC,CalculoPrevisorOC_mx,DscJustificativaPMO,dscjustificativapmo
48,28060,1,Não,Não,ME,0,NaT,2023-05-20,2023-02-14,Estágio Atual das Obras,Null
88,29116,2,Não,Não,ME,0,NaT,2023-08-09,2023-11-06,Estágio Atual das Obras,Null
128,30196,3,Não,Não,IO,0,NaT,2024-01-24,2024-04-02,Estágio Atual das Obras,Null
133,30264,2,Não,Não,IO,0,NaT,2024-01-24,2024-04-02,Estágio Atual das Obras,Null
134,30355,2,Não,Não,OUT,1,NaT,2025-05-01,2025-05-18,Estágio Atual das Obras,Null
...,...,...,...,...,...,...,...,...,...,...,...
56893,51294,132,Não,Não,OUT,1,NaT,2024-06-17,2024-04-09,Estágio Atual das Obras,Null
56910,51294,133,Não,Não,OUT,1,NaT,2024-06-17,2024-04-09,Estágio Atual das Obras,Null
56926,51294,134,Não,Não,OUT,1,NaT,2024-06-17,2024-04-09,Estágio Atual das Obras,Null
56994,51294,131,Não,Não,OUT,1,NaT,2024-06-17,2024-04-09,Estágio Atual das Obras,Null


In [ ]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga","DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada"]
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols]

In [ ]:
# vInputCronograma tbm implementado,DATASREAL (LABEL="Idempreendimento" AS Idempreendimento)
cols = ["DatRealizacaoIII","DatRealizacaoV","DatRealizacaoXIV","DatRealizacaoXII","IdeUsinaOutorga","DatRealizacaoIV","DatRealizacaoIX","DatRealizacaoXI","DatRealizacaoX","DthEnvio","DatRealizacaoVI","DatRealizacaoII","CodCeg","DatRealizacaoXIII","DatRealizacaoVII"]
rename_cols = {'DatRealizacaoIII': 'IO_real','DatRealizacaoXIV': 'Conc_Trans_REAL','DatRealizacaoXII': 'Comiss_Real','DatRealizacaoIX': 'DesvRio_real','DatRealizacaoXI': 'Ench_Real','DthEnvio': 'Data_Envio','DatRealizacaoII': 'canteiroReal','DatRealizacaoXIII': 'IOTrans_Real','DatRealizacaoVII': 'ME_Real_conc_eol'}
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].drop_duplicates()[cols].rename(columns=rename_cols)
vcronograma["classe"] = vcronograma.CodCeg.str.slice(3,5)
vcronograma["CC_real"] = pd.to_datetime(pd.NA)
vcronograma.loc[vcronograma.classe == "PH","CC_real"] = vcronograma.DatRealizacaoIV
vcronograma.loc[vcronograma.classe == "CV","CC_real"] = vcronograma.DatRealizacaoV
vcronograma["ME_real"] = vcronograma.DatRealizacaoX
vcronograma.loc[vcronograma.classe == "CV","ME_real"] = vcronograma.DatRealizacaoVI